In [1]:
from statsbombpy import sb
import math
import pandas as pd

In [2]:
columns = [
    'id',
    'match_id',
    'index',
    'minute',
    'second',
    'possession',
    'possession_team_id',
    'possession_team',
    'type',
    'player_id',
    'player',
    'position',
    'team_id',
    'team',
    'location',
    'pass_end_location',
    'pass_outcome',
    'ball_receipt_outcome',
    'pass_recipient_id',
    'pass_recipient',
    'carry_end_location',
    'dribble_outcome',
    'foul_won_advantage',
    'related_events',
    'under_pressure'
]

events = sb.competition_events(
    country="Germany",
    division= "1. Bundesliga",
    season="2015/2016",
    gender="male"
)[columns]

events = events[events.match_id.isin([3890305, 3890561, 3890511, 3890515, 3890263, 3890261])]

events.head()

/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data acce

,id,match_id,index,minute,second,possession,possession_team_id,possession_team,type,player_id,...,pass_end_location,pass_outcome,ball_receipt_outcome,pass_recipient_id,pass_recipient,carry_end_location,dribble_outcome,foul_won_advantage,related_events,under_pressure
0,41bd60ac-9b2c-4cb8-85aa-23ae71825c1e,3890561,1,0,0,1,175,Hoffenheim,Starting XI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fbca533d-f3f4-4a86-b4a3-4fcae63592cf,3890561,2,0,0,1,175,Hoffenheim,Starting XI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20ae06cd-9bf8-4797-a14a-2ad74773e772,3890511,1,0,0,1,173,Hertha Berlin,Starting XI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,e27699cf-9ddb-41b5-a408-e0e682931688,3890511,2,0,0,1,173,Hertha Berlin,Starting XI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,d4d15ebe-fa14-41a5-94cd-febba3555581,3890515,1,0,0,1,171,Hamburger SV,Starting XI,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
matches = sb.matches(competition_id=9, season_id=27)[['match_id', 'competition', 'season', 'match_date', 'home_team', 'home_score', 'away_score', 'away_team']]

matches = matches[matches.match_id.isin([3890305, 3890561, 3890511, 3890515, 3890263, 3890261])]

matches

/Users/prateek/miniconda3/envs/pass_reception_and_transition/lib/python3.12/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(


,match_id,competition,season,match_date,home_team,home_score,away_score,away_team
0,3890561,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04
2,3890511,Germany - 1. Bundesliga,2015/2016,2016-04-08,Hertha Berlin,2,2,Hannover 96
3,3890515,Germany - 1. Bundesliga,2015/2016,2016-04-09,Hamburger SV,1,2,Darmstadt 98
263,3890305,Germany - 1. Bundesliga,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg
301,3890263,Germany - 1. Bundesliga,2015/2016,2015-08-15,FSV Mainz 05,0,1,Ingolstadt
303,3890261,Germany - 1. Bundesliga,2015/2016,2015-08-15,Borussia Dortmund,4,0,Borussia Mönchengladbach


In [4]:
def pass_start_condition(x):
    return events[events.id == list(filter(lambda x: events[events.id == x].type.item() == 'Pass', x))[0]].location.item()[0] < 60

In [5]:
ball_receipts = events[(events.type == 'Ball Receipt*') & (events.ball_receipt_outcome.isna()) & (events.position.str.contains('Midfield'))]
ball_receipts = ball_receipts[ball_receipts.location.apply(lambda x: 20 < x[0] < 60)]
ball_receipts = ball_receipts[ball_receipts.related_events.apply(pass_start_condition)]
ball_receipts = pd.merge(left = ball_receipts, right = matches, how = 'left', on = 'match_id')

ball_receipts.head()

,id,match_id,index,minute,second,possession,possession_team_id,possession_team,type,player_id,...,foul_won_advantage,related_events,under_pressure,competition,season,match_date,home_team,home_score,away_score,away_team
0,69e3b0a7-5847-40db-91d5-0e24fb150f63,3890561,9,0,2,2,175,Hoffenheim,Ball Receipt*,6039.0,...,NaN,[77e2ddaf-6de3-49e7-a318-7d765799b543],NaN,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04
1,d843ba4b-de1e-4e81-a71d-0f3781c8f3fa,3890561,63,0,56,3,181,Schalke 04,Ball Receipt*,3053.0,...,NaN,[57acd29e-7ac3-45e9-acaa-cfa96d74c10f],NaN,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04
2,45271883-07e0-446f-ba83-3ddc3e678def,3890561,69,1,4,3,181,Schalke 04,Ball Receipt*,5242.0,...,NaN,[e5126f2f-a414-4954-a607-927f102fe5bb],NaN,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04
3,7d08b692-1f2e-4b44-be64-7d3e04632ab9,3890561,86,1,14,3,181,Schalke 04,Ball Receipt*,5242.0,...,NaN,[cbeb8953-feeb-45d0-84ce-a0416be4eeba],NaN,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04
4,0a325d10-463e-4e70-86bd-1d2ae118864c,3890561,92,1,19,3,181,Schalke 04,Ball Receipt*,5242.0,...,NaN,[7bb92ebe-6834-4cdf-9888-5dc19f0a53e3],NaN,Germany - 1. Bundesliga,2015/2016,2016-05-14,Hoffenheim,1,4,Schalke 04


In [51]:
data = dict()
for receipt in ball_receipts[(ball_receipts.match_id == 3890305)].itertuples():
    event_dict = dict()
    event_count = 0
    final_location_x = receipt.location[0]
    players = set()
    dribble = False
    last_carry_start_location = receipt.location
    last_carry_distance = 0
    possession_events = events[(events.match_id == receipt.match_id) & (events.possession == receipt.possession) & (events['index'] > receipt.index)].sort_values('index')
    for event in possession_events.itertuples():
        if final_location_x >= 80 or event_count == 4 or len(players) == 2:
            break
        else:
            if event.type == 'Pass' and pd.isna(event.pass_outcome):
                event_count += 1
                final_location_x = event.pass_end_location[0]
                players.add(event.player_id)
            elif dribble == False and event.type == 'Carry':
                if math.dist(event.location, event.carry_end_location) > 5.46807:
                    event_count += 1
                final_location_x = event.carry_end_location[0]
                players.add(event.player_id)
                last_carry_start_location = event.location
                last_carry_distance = math.dist(event.location, event.carry_end_location)
            elif event.type in ['Dispossessed', 'Miscontrol', 'Interception', 'Clearance', 'Block', 'Shot'] \
            or (event.type == 'Pass' and pd.notna(event.pass_outcome)) \
            or (event.type == 'Dribble' and event.dribble_outcome == 'Incomplete') \
            or (event.type == 'Foul Won' and pd.isna(event.foul_won_advantage)):
                break
            elif event.type == 'Dribble' and event.dribble_outcome == 'Complete':
                dribble = True
            elif dribble == True and event.type == 'Carry':
                if last_carry_distance <= 5.46807 and math.dist(last_carry_start_location, event.carry_end_location) > 5.46807:
                    event_count += 1
                final_location_x = event.carry_end_location[0]
                dribble = False
                last_carry_distance = math.dist(last_carry_start_location, event.carry_end_location)
            else:
                continue
    event_dict['successful_transition'] = True if final_location_x >= 80 else False
    event_dict['transition_final_location_x'] = final_location_x
    event_dict['transition_event_count'] = event_count 
    event_dict['transition_players_involved'] = len(players)
    data[receipt.id] = event_dict

In [52]:
pd.DataFrame.from_dict(data=data, orient='index').reset_index().rename(columns = {'index': 'id'})

,id,successful_transition,transition_final_location_x,transition_event_count,transition_players_involved
0,12a91213-f134-458d-834f-0d15dfe02709,False,50.0,1,2
1,bc14b45f-acfe-4df3-a4a8-12c5be1a1ffd,False,57.9,1,2
2,d623af6a-c946-422b-8e5e-7c3a6715bfff,False,41.2,0,0
3,7be3359c-454a-405f-b06d-4a59935202ad,False,23.2,2,2
4,ee40f522-d063-46a1-80a5-b74921c192bd,False,21.5,1,2
...,...,...,...,...,...
146,f16c1961-c2d5-4ca3-8e44-ca73a57493f1,True,90.3,2,1
147,c8ec5d08-7960-461c-bf0b-c8b1332da728,False,79.2,2,2
148,453c357f-8b31-4a84-9756-c5a128f7775e,False,50.1,1,2
149,8032760f-dd5a-4e70-ae3b-a20238f55c18,False,54.8,2,2


In [53]:
results = pd.merge(
    left = ball_receipts[(ball_receipts.match_id == 3890305)],
    right = pd.DataFrame.from_dict(data=data, orient='index').reset_index().rename(columns = {'index': 'id'}),
    how = 'left',
    on = 'id'
)

results.head()

,id,match_id,index,minute,second,possession,possession_team_id,possession_team,type,player_id,...,season,match_date,home_team,home_score,away_score,away_team,successful_transition,transition_final_location_x,transition_event_count,transition_players_involved
0,12a91213-f134-458d-834f-0d15dfe02709,3890305,8,0,3,2,169,Bayern Munich,Ball Receipt*,20133.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,False,50.0,1,2
1,bc14b45f-acfe-4df3-a4a8-12c5be1a1ffd,3890305,11,0,5,2,169,Bayern Munich,Ball Receipt*,8206.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,False,57.9,1,2
2,d623af6a-c946-422b-8e5e-7c3a6715bfff,3890305,28,0,28,3,179,Wolfsburg,Ball Receipt*,3545.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,False,41.2,0,0
3,7be3359c-454a-405f-b06d-4a59935202ad,3890305,53,0,47,4,169,Bayern Munich,Ball Receipt*,5208.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,False,23.2,2,2
4,ee40f522-d063-46a1-80a5-b74921c192bd,3890305,107,1,48,6,179,Wolfsburg,Ball Receipt*,3545.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,False,21.5,1,2


In [54]:
results.columns

Index(['id', 'match_id', 'index', 'minute', 'second', 'possession',
       'possession_team_id', 'possession_team', 'type', 'player_id', 'player',
       'position', 'team_id', 'team', 'location', 'pass_end_location',
       'pass_outcome', 'ball_receipt_outcome', 'pass_recipient_id',
       'pass_recipient', 'carry_end_location', 'dribble_outcome',
       'foul_won_advantage', 'related_events', 'under_pressure', 'competition',
       'season', 'match_date', 'home_team', 'home_score', 'away_score',
       'away_team', 'successful_transition', 'transition_final_location_x',
       'transition_event_count', 'transition_players_involved'],
      dtype='object')

In [55]:
results[(results.successful_transition == True) & (results.possession_team_id == 169)]

,id,match_id,index,minute,second,possession,possession_team_id,possession_team,type,player_id,...,season,match_date,home_team,home_score,away_score,away_team,successful_transition,transition_final_location_x,transition_event_count,transition_players_involved
9,f7855b27-43a1-407a-8e56-5bdec91bf516,3890305,334,5,52,16,169,Bayern Munich,Ball Receipt*,5208.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,True,84.8,2,2
12,87e41e9d-4f53-404f-bc8d-610028f5efd8,3890305,376,6,26,18,169,Bayern Munich,Ball Receipt*,5208.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,True,91.8,3,2
27,f3376741-c988-4a25-93d3-540205a8bccb,3890305,855,15,40,40,169,Bayern Munich,Ball Receipt*,5208.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,True,82.0,2,2
40,6d50e016-5d9f-4770-a86e-4353e1a92a52,3890305,1323,25,20,60,169,Bayern Munich,Ball Receipt*,5208.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,True,80.5,1,1
48,52095e72-ec10-4501-96f7-e54254ae2a93,3890305,1696,34,37,78,169,Bayern Munich,Ball Receipt*,5208.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,True,87.0,2,1
50,389d071d-4f8d-457f-a907-505172c0e573,3890305,1751,35,35,78,169,Bayern Munich,Ball Receipt*,5208.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,True,83.0,2,1
71,158bb3b0-da93-43af-9f53-e069f373e92c,3890305,2625,53,54,115,169,Bayern Munich,Ball Receipt*,8206.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,True,85.2,1,1
124,36ed849f-8a24-4d29-8e26-0edffb4ebbc2,3890305,3761,80,7,158,169,Bayern Munich,Ball Receipt*,5562.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,True,94.2,2,1
146,f16c1961-c2d5-4ca3-8e44-ca73a57493f1,3890305,4178,87,19,175,169,Bayern Munich,Ball Receipt*,5562.0,...,2015/2016,2015-09-22,Bayern Munich,5,1,Wolfsburg,True,90.3,2,1


In [58]:
events[(events.match_id == 3890305) & (events.possession == 60)].sort_values('index')

,id,match_id,index,minute,second,possession,possession_team_id,possession_team,type,player_id,...,pass_end_location,pass_outcome,ball_receipt_outcome,pass_recipient_id,pass_recipient,carry_end_location,dribble_outcome,foul_won_advantage,related_events,under_pressure
257391,9d01f882-be43-4428-a0c0-750f1d8ecc49,3890305,1316,25,14,60,169,Bayern Munich,Pass,5837.0,...,"[57.3, 64.4]",NaN,NaN,19295.0,Philipp Lahm,NaN,NaN,NaN,[748e519e-bab7-4d36-86fb-3c7195a5957f],NaN
542210,748e519e-bab7-4d36-86fb-3c7195a5957f,3890305,1317,25,15,60,169,Bayern Munich,Ball Receipt*,19295.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[9d01f882-be43-4428-a0c0-750f1d8ecc49],NaN
768647,fe02439e-cb40-4ee6-959e-796cf2fd1470,3890305,1318,25,15,60,169,Bayern Munich,Carry,19295.0,...,NaN,NaN,NaN,NaN,NaN,"[57.8, 63.6]",NaN,NaN,"[748e519e-bab7-4d36-86fb-3c7195a5957f, b304454...",NaN
257392,b3044541-864b-460a-8961-beb73e9a74a7,3890305,1319,25,16,60,169,Bayern Munich,Pass,19295.0,...,"[48.2, 36.1]",NaN,NaN,8220.0,David Olatukunbo Alaba,NaN,NaN,NaN,[4b90d5a3-12e6-4d22-b0ee-0fee4bbb6395],NaN
542211,4b90d5a3-12e6-4d22-b0ee-0fee4bbb6395,3890305,1320,25,18,60,169,Bayern Munich,Ball Receipt*,8220.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[b3044541-864b-460a-8961-beb73e9a74a7],NaN
768648,1671d8a2-4f47-4ae7-8bb3-ae536930af4e,3890305,1321,25,18,60,169,Bayern Munich,Carry,8220.0,...,NaN,NaN,NaN,NaN,NaN,"[48.0, 33.2]",NaN,NaN,"[4b90d5a3-12e6-4d22-b0ee-0fee4bbb6395, 4b995dd...",NaN
257393,4b995dd4-265f-45c9-82ab-519f03d27042,3890305,1322,25,19,60,169,Bayern Munich,Pass,8220.0,...,"[54.1, 20.4]",NaN,NaN,5208.0,Thiago Alcântara do Nascimento,NaN,NaN,NaN,[6d50e016-5d9f-4770-a86e-4353e1a92a52],NaN
542212,6d50e016-5d9f-4770-a86e-4353e1a92a52,3890305,1323,25,20,60,169,Bayern Munich,Ball Receipt*,5208.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[4b995dd4-265f-45c9-82ab-519f03d27042],NaN
768649,69234fed-0c6c-41a5-a3b0-542b7b19d828,3890305,1324,25,20,60,169,Bayern Munich,Carry,5208.0,...,NaN,NaN,NaN,NaN,NaN,"[55.9, 19.6]",NaN,NaN,"[34f87a34-f652-43a5-b3c5-f7ae16503e3e, 6d50e01...",NaN
257394,34f87a34-f652-43a5-b3c5-f7ae16503e3e,3890305,1325,25,21,60,169,Bayern Munich,Pass,5208.0,...,"[80.5, 2.6]",NaN,NaN,15651.0,Juan Bernat Velasco,NaN,NaN,NaN,[eaed867b-f521-419b-9dee-314f39f16e58],NaN
